In [1]:
import pandas as pd                       
import numpy as np                       
import tensorflow as tf                       
from keras.models import Sequential      

from keras.preprocessing import image            
from keras.preprocessing.image import ImageDataGenerator  

from tensorflow.keras.optimizers import Adam  
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout     
from keras.utils import np_utils  

In [2]:
emotion_data = pd.read_csv('coords_train.csv')
emotion_data_test = pd.read_csv('coords_test.csv')
print(emotion_data_test)

      class        x1        y1        z1   v1        x2        y2        z2  \
0         0  0.499253  0.752961 -0.034682  0.0  0.559338  0.693915 -0.144536   
1         0  0.689811  0.742115 -0.079950  0.0  0.724580  0.616124 -0.141577   
2         0  0.378256  0.506817 -0.086138  0.0  0.344555  0.419925 -0.121530   
3         0  0.522812  0.712983 -0.101182  0.0  0.445895  0.615441 -0.156803   
4         0  0.405928  0.552301 -0.082483  0.0  0.395725  0.438942 -0.167460   
...     ...       ...       ...       ...  ...       ...       ...       ...   
6632      6  0.528955  0.762481 -0.097065  0.0  0.526803  0.624564 -0.179154   
6633      6  0.480809  0.586866 -0.104279  0.0  0.475075  0.464702 -0.215600   
6634      6  0.516132  0.698601 -0.080681  0.0  0.513222  0.597312 -0.162645   
6635      6  0.544261  0.851070 -0.064527  0.0  0.527826  0.731073 -0.204287   
6636      6  0.500083  0.721104 -0.088762  0.0  0.519381  0.618235 -0.179013   

       v2        x3  ...      z466  v46

In [5]:
for index, row in emotion_data.iterrows():  
    row[1:1873]
row[1:1873]

x1      0.477492
y1      0.664674
z1     -0.153198
v1      0.000000
x2      0.500189
          ...   
v467    0.000000
x468    0.800666
y468    0.337344
z468    0.111730
v468    0.000000
Name: 26556, Length: 1872, dtype: float64

In [6]:
X_train, y_train, X_test, y_test = [], [], [], []   

for index, row in emotion_data.iterrows():  
    X_train.append(np.array(row[1:1873]))
    y_train.append(row['class'])

for index, row in emotion_data_test.iterrows():  
    X_test.append(np.array(row[1:1873]))   
    y_test.append(row['class'])

In [7]:
print(X_train[0])
#print(y_train)
#print(type(X_test))
#print(type(y_test))

[ 0.48071757  0.86503303  0.0117513  ...  0.49864233 -0.05259559
  0.        ]


In [8]:
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

#print(type(X_test))
#print(type(y_test))

In [9]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)   
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)

#print(X_test)
#print(y_test)

ValueError: cannot reshape array of size 49714704 into shape (26557,48,48,1)

In [ ]:
#VGG11 Model
model = Sequential()

model.add(Conv2D(input_shape=(X_train.shape[1:]),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
emotion_model_info = model.fit(
    X_train,
    y_train,
    batch_size= 32,
    epochs= 30,
    verbose=1,
    validation_data=(X_test, y_test),
    shuffle=True)

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
  model.save_weights("model.h5")

print("Saved model to disk")